<a href="https://colab.research.google.com/github/khadidja2005/Pytorch_practice/blob/main/digits_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [47]:
# calculating mean and std for transform

datasett = datasets.MNIST(root = "/data" , train = True , download = True ,transform = transforms.ToTensor())
loader = DataLoader(datasett, batch_size=60000, shuffle=False)
data = next(iter(loader))[0]
mean = data.mean().item()
std = data.std().item()
print(f"mean : {mean} , std : {std}")

mean : 0.13066047430038452 , std : 0.30810782313346863


In [48]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = mean , std= std)
])
train_dataset = datasets.MNIST(root = "/data" , train = True , download = True , transform = transform)
test_dataset = datasets.MNIST(root = "/data" , train = False , download = True , transform = transform)

train_loader = DataLoader(train_dataset , batch_size=64 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size= 1000 , shuffle=False)


In [49]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("No GPU available, using the CPU instead.")

GPU is available and being used.


In [50]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel , self).__init__()
    self.conv1 = nn.Conv2d(1 , 32 , kernel_size=3)
    self.conv2 = nn.Conv2d(32 , 64 , kernel_size=3)
    self.fc1 = nn.Linear(64*12*12 , 128)
    self.fc2 = nn.Linear(128 , 10)
  def forward(self , x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x , 2)
    x = torch.flatten(x , 1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x , dim = 1)
model = CNNModel().to(device)

In [51]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=0.001)


In [52]:
num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  for indx , (data , target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs , target)
    loss.backward()
    optimizer.step()
    if indx % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Batch {indx}/{len(train_loader)}, Loss: {loss.item()}')

Epoch 1/10, Batch 0/938, Loss: 2.3004374504089355
Epoch 1/10, Batch 100/938, Loss: 0.075043223798275
Epoch 1/10, Batch 200/938, Loss: 0.08667907118797302
Epoch 1/10, Batch 300/938, Loss: 0.0908079445362091
Epoch 1/10, Batch 400/938, Loss: 0.10597997903823853
Epoch 1/10, Batch 500/938, Loss: 0.03666553646326065
Epoch 1/10, Batch 600/938, Loss: 0.16997814178466797
Epoch 1/10, Batch 700/938, Loss: 0.046140190213918686
Epoch 1/10, Batch 800/938, Loss: 0.052586689591407776
Epoch 1/10, Batch 900/938, Loss: 0.03759946674108505
Epoch 2/10, Batch 0/938, Loss: 0.014973055571317673
Epoch 2/10, Batch 100/938, Loss: 0.003984479233622551
Epoch 2/10, Batch 200/938, Loss: 0.07035911828279495
Epoch 2/10, Batch 300/938, Loss: 0.04827844724059105
Epoch 2/10, Batch 400/938, Loss: 0.0419294610619545
Epoch 2/10, Batch 500/938, Loss: 0.03147599846124649
Epoch 2/10, Batch 600/938, Loss: 0.0008587393094785511
Epoch 2/10, Batch 700/938, Loss: 0.011603101156651974
Epoch 2/10, Batch 800/938, Loss: 0.0185509659349

In [53]:
def calculate_accuracy(pred, target):
    correct = pred.eq(target.view_as(pred)).sum().item()
    return correct

In [56]:
test_loss = 0
correct = 0
model.eval()
with torch.no_grad():
  for data , labels in test_loader:
    data, target = data.to(device), labels.to(device)
    output = model(data)
    #print(output.shape, target.shape)
    test_loss += criterion(output , target).item()
    pred = output.argmax(dim=1 , keepdim = True)
    correct += calculate_accuracy(pred , target)
test_loss /= len(test_loader.dataset)
accuracy = correct / len(test_loader.dataset)

print(f"accuracy : {accuracy}")
print(f"loss : {test_loss}")

accuracy : 0.9908
loss : 4.261492900550366e-05
